In [1]:
import tkinter as tk
import customtkinter as ctk
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn as sk
from sklearn.preprocessing import LabelEncoder
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from sklearn import tree
from sklearn.model_selection import train_test_split
import datetime



In [2]:
app = tk.Tk()

if app.call("tk", "scaling") == 0:
    # "dark" for MacOS
    appearance_mode = "light"
else:
    # "light" for Windows
    appearance_mode = "dark"

app.destroy()


In [3]:


def run_program():
    global image
    run_button.pack_forget()
    df = pd.read_csv('IndianWeatherRepository.csv')
    col_to_remove=['country','latitude','longitude','timezone','last_updated_epoch','temperature_celsius','wind_mph','wind_direction','pressure_in','precip_in','feels_like_celsius','feels_like_fahrenheit','visibility_miles','gust_mph','uv_index','gust_kph','sunrise','sunset','moonrise','moonset','moon_phase','moon_illumination','location_name','air_quality_Carbon_Monoxide', 'air_quality_Ozone','air_quality_Nitrogen_dioxide','air_quality_Sulphur_dioxide','air_quality_PM2.5','air_quality_PM10','air_quality_us-epa-index','air_quality_gb-defra-index']
    df = df.drop(columns=col_to_remove)
    condition=df['region']=='Andaman and Nicobar Islands'
    df = df[condition]
    df['humidity'] = df['humidity'].astype('float64')
    df['wind_degree'] = df['wind_degree'].astype('float64')
    df['cloud'] = df['cloud'].astype('float64')
    encoder = LabelEncoder()
    df['condition_text'] = encoder.fit_transform(df['condition_text'])
    df['condition_text'] = df['condition_text'].astype(int)
    df['last_updated'] = pd.to_datetime(df['last_updated'])
    df.set_index('last_updated', inplace=True)
    parameters = ['temperature_fahrenheit', 'wind_kph', 'wind_degree', 'pressure_mb',
              'precip_mm', 'humidity', 'cloud', 'visibility_km']
    predictions_df = pd.DataFrame()
    for param in parameters:
        train_data = df[:-7][param]
        test_data = df[-7:][param]

    # Fit an ARIMA model
        model = ARIMA(train_data, order=(5, 2, 2))  # Adjust the order as needed
        model_fit = model.fit()
        predictions = model_fit.forecast(steps=7)
        predictions_df[param] = predictions
        
    numerical_columns = df.select_dtypes(include=['float64', 'int64']).columns
    scaler = MinMaxScaler()
    df[numerical_columns] = scaler.fit_transform(df[numerical_columns])
    X = df.iloc[:, [1,3,4,5,6,7,8,9]].values
    y = df.iloc[:, 2].values
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
    clf = tree.DecisionTreeClassifier(random_state=18)
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    numerical_columns = predictions_df.select_dtypes(include=['float64', 'int64']).columns
    scaler = MinMaxScaler()
    predictions_df[numerical_columns] = scaler.fit_transform(predictions_df[numerical_columns])
    pred=clf.predict(predictions_df)
    decoded_labels = encoder.inverse_transform(pred) 

    selected_df = df.iloc[1, [1,3,7]]
    name=pd.to_datetime(selected_df.name)

    
    day = name.day_name()
    
    frame = tk.Frame(app, bg="white")
    frame.pack(side="top",fill="both")
    label = tk.Label(frame, text="DAY- 1", bg="white")
    label.pack()
    label = tk.Label(frame, text=decoded_labels[0].upper(), bg="white")
    label.pack()

    label_frame=tk.Frame(frame, bg ="white")
    label_frame.pack(side=tk.LEFT)
    label = tk.Label(label_frame, text=selected_df, bg="white")
    label.pack(side=tk.LEFT)
    canvas = tk.Canvas(frame, width=200, height=200, bg="white")
    canvas.pack(side=tk.RIGHT)
    canvas.config(highlightthickness=1, highlightbackground="white")
    # Load the image
    image = tk.PhotoImage(file="cloudy.png")
    # Add the image to the canvas
    canvas.create_image(100, 100, image=image)

    for i in range(6):
        frame = tk.Frame(app, bg="white", width=40,height=20)
        frame.pack(side=tk.LEFT, padx=10, pady=5, expand=True)
    
        label = tk.Label(frame, text="DAY- "+str(i+2), bg="white")
        label.pack(pady=5)
        label = tk.Label(frame, text=decoded_labels[i+1], bg="white")
        label.pack(pady=5)
pass




In [4]:
def gui_buttons():
    
    pass

In [5]:
ctk.set_appearance_mode("System")
ctk.set_default_color_theme("blue")

app = tk.Tk()
app.geometry("720x480")
app.title("RainCheck")
# Define the RGB color (replace the numbers with your desired RGB values)
rgb_color = (50, 100, 150)
# Convert the RGB color to Hex
hex_color = "#{:02x}{:02x}{:02x}".format(*rgb_color)
# Set the background color to the Hex color
app.configure(bg=hex_color)
heading = tk.Label(app, text="WEATHER FORECAST",font=("Helvetica", 18, "bold"),anchor="e")
heading.configure(bg=hex_color)
heading.pack()
run_button = tk.Button(app, text="Get Weather", command=run_program)
run_button.pack()
app.mainloop()


c:\Users\HP\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:581: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  warnings.warn('A date index has been provided, but it has no'
c:\Users\HP\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:581: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  warnings.warn('A date index has been provided, but it has no'
c:\Users\HP\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:581: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  warnings.warn('A date index has been provided, but it has no'
c:\Users\HP\anaconda3\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros a